# High-level CNTK Example

In [1]:
import numpy as np
import os
import sys
import cntk
from cntk.layers import Convolution2D, MaxPooling, Dense, Dropout
from common.params import *
from common.utils import *

In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("CNTK: ", cntk.__version__)
print("GPU: ", get_gpu_name())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.1
CNTK:  2.1


In [3]:
def create_symbol():
    # Weight initialiser from uniform distribution
    # Activation (unless states) is None
    with cntk.layers.default_options(init = cntk.glorot_normal(), activation = cntk.relu):
        x = Convolution2D(filter_shape=(3, 3), num_filters=50, pad=True)(features)
        x = Convolution2D(filter_shape=(3, 3), num_filters=50, pad=True)(x)
        x = MaxPooling((2, 2), strides=(2, 2), pad=False)(x)
        x = Dropout(0.25)(x)

        x = Convolution2D(filter_shape=(3, 3), num_filters=100, pad=True)(x)
        x = Convolution2D(filter_shape=(3, 3), num_filters=100, pad=True)(x)
        x = MaxPooling((2, 2), strides=(2, 2), pad=False)(x)
        x = Dropout(0.25)(x)    
        
        x = Dense(512)(x)
        x = Dropout(0.5)(x)
        x = Dense(N_CLASSES, activation=None)(x)
        return x

In [4]:
def init_model(m):
    # Loss (dense labels); check if support for sparse labels
    loss = cntk.cross_entropy_with_softmax(m, labels)
    # Classification error
    label_error = cntk.classification_error(m, labels)
    # Init trainer object
    lr_schedule = cntk.learning_rate_schedule(LR, cntk.UnitType.minibatch) 
    m_schedule = cntk.momentum_schedule(MOMENTUM)
    
    # Momentum SGD
    # https://github.com/Microsoft/CNTK/blob/master/Manual/Manual_How_to_use_learners.ipynb
    # unit_gain=False: momentum_direction = momentum*old_momentum_direction + gradient
    # if unit_gain=True then ...(1-momentum)*gradient
    
    #def momentum_sgd(parameters, lr, momentum, unit_gain=default_unit_gain_value(),
    #             l1_regularization_weight=0.0, l2_regularization_weight=0.0,
    #             gaussian_noise_injection_std_dev=0.0, gradient_clipping_threshold_per_sample=np.inf,
    #             gradient_clipping_with_truncation=True, use_mean_gradient=default_use_mean_gradient_value()):
    learner = cntk.momentum_sgd(m.parameters, lr=lr_schedule, momentum=m_schedule, unit_gain=False)
    trainer = cntk.Trainer(m, (loss, label_error), [learner])
    return trainer

In [5]:
%%time
# Data into format for library
#x_train, x_test, y_train, y_test = mnist_for_library(channel_first=True, one_hot=True)
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True, one_hot=True)
# CNTK format
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Done.
Extracting files...
Done.
Preparing train set...
Preparing test set...
Done.
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000, 10) (10000, 10)
float32 float32 float32 float32
CPU times: user 2.92 s, sys: 1.58 s, total: 4.5 s
Wall time: 43.2 s


In [6]:
%%time
# Placeholders
features = cntk.input_variable((3, 32, 32), np.float32)
labels = cntk.input_variable(N_CLASSES, np.float32)
# Load symbol
sym = create_symbol()

CPU times: user 24 ms, sys: 48 ms, total: 72 ms
Wall time: 68.6 ms


In [7]:
%%time
trainer = init_model(sym)

CPU times: user 88 ms, sys: 156 ms, total: 244 ms
Wall time: 254 ms


In [8]:
%%time
# Train model
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        trainer.train_minibatch({features: data, labels: label})
    # Log (this is just last batch in epoch, not average of batches)
    eval_error = trainer.previous_minibatch_evaluation_average
    print("Epoch %d  |  Accuracy: %.6f" % (j+1, (1-eval_error)))

Epoch 1  |  Accuracy: 0.453125
Epoch 2  |  Accuracy: 0.484375
Epoch 3  |  Accuracy: 0.578125
Epoch 4  |  Accuracy: 0.640625
Epoch 5  |  Accuracy: 0.578125
Epoch 6  |  Accuracy: 0.750000
Epoch 7  |  Accuracy: 0.687500
Epoch 8  |  Accuracy: 0.750000
Epoch 9  |  Accuracy: 0.812500
Epoch 10  |  Accuracy: 0.781250
CPU times: user 2min 16s, sys: 27.7 s, total: 2min 44s
Wall time: 2min 48s


In [9]:
%%time
# Predict and then score accuracy
# Apply softmax since that is only applied at training
# with cross-entropy loss
z = cntk.softmax(sym)
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = np.argmax(y_test[:n_samples], axis=-1)
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    predicted_label_probs = z.eval({features : data})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = np.argmax(predicted_label_probs, axis=-1)
    c += 1

CPU times: user 1.46 s, sys: 544 ms, total: 2.01 s
Wall time: 3.25 s


In [10]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.76702724359
